In [22]:
import pandas as pd
import numpy as np
from pgmpy.estimators import K2Score
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, BayesianEstimator
import random

def load_data(DATA_CSV):
    D = pd.read_csv(DATA_CSV)
    V = D.columns
    N = len(D.index)
    V_CARD = {v: len(D[v].unique()) for v in V}
    #print(f'ARQUIVO: {D}')
    #print(f'VARIÁVEIS: {V}')
    #print(f'NÚMERO DE AMOSTRAS: {N}')
    #print(f'MAPEAMENTO DAS VARIÁVEIS COM NÚMEROS DE VALORES ÚNICOS: {V_CARD}')
    return D, V, N, V_CARD

def calcular_k2(D):
    k2score = K2Score(D)
    return k2score

def estimar_modelo(D, scoring_method):
    #print(f'Ordenação dentro do estimar_modelo: {D}')
    print(f'Lista de variáveis dentro do estimar_modelo: {V}')
    estimator_k2 = HillClimbSearch(D)
    max_possible_edges = len(V) * (len(V) - 1) / 2
    max_iter = min(max_possible_edges, 1000)
    best_model = estimator_k2.estimate(scoring_method=K2Score(D), tabu_length=50, max_indegree=4, max_iter=max_iter)
    k2score = calcular_k2(D)
    k2_score = k2score.score(best_model)
    print(f'Valor do score K2: {k2_score}')
    return best_model

def tabular_cpd(best_model, D):
    bayesian_network = BayesianModel(best_model)
    estimator = BayesianEstimator(bayesian_network, D)
    cpds = []
    for node in bayesian_network.nodes():
        cpd = estimator.estimate_cpd(node)
        cpds.append(cpd)
    return cpds, bayesian_network

def generate_shuffled_csv(DATA_CSV):
    D = pd.read_csv(DATA_CSV)
    D_shuffled_columns = D.sample(frac=1, axis=1)
    D_shuffled_columns.to_csv('data_shuffled.csv', index=False)

# Caminho do arquivo CSV original
DATA_CSV = 'alarm.csv'

# Gerar um novo arquivo CSV com os dados embaralhados
generate_shuffled_csv(DATA_CSV)

# Carregar dados
D, V, N, V_CARD = load_data('data_shuffled.csv')

# Estima a estrutura do modelo com o K2
#print(f'Ordem das variáveis antes de estimar: {D}')
print(f'Ordem das variáveis antes de estimar: {V}')
best_model = estimar_modelo(D, 'k2score')
print(f'Ordem das variáveis: {V}')
print(f'Melhor modelo: {best_model}')

# Exibe a estrutura do modelo
structure = (best_model.edges)
print(f'Estrutura da rede: {structure}')

# Estima as CPDs e passa a bayesian_network
cpds, bayesian_network = tabular_cpd(best_model, D)
print(f'CPDs: {cpds}')
print(f'Bayesian Network: {bayesian_network}')


Ordem das variáveis antes de estimar: Index(['MINVOLSET', 'STROKEVOLUME', 'PVSAT', 'INSUFFANESTH', 'HRSAT', 'EXPCO2',
       'INTUBATION', 'BP', 'HR', 'PCWP', 'DISCONNECT', 'HISTORY', 'HREKG',
       'LVEDVOLUME', 'VENTMACH', 'ERRLOWOUTPUT', 'CATECHOL', 'VENTALV',
       'ANAPHYLAXIS', 'ARTCO2', 'PRESS', 'SHUNT', 'SAO2', 'ERRCAUTER',
       'LVFAILURE', 'HYPOVOLEMIA', 'FIO2', 'KINKEDTUBE', 'TPR', 'CO',
       'VENTLUNG', 'MINVOL', 'VENTTUBE', 'PULMEMBOLUS', 'PAP', 'HRBP', 'CVP'],
      dtype='object')
Lista de variáveis dentro do estimar_modelo: Index(['MINVOLSET', 'STROKEVOLUME', 'PVSAT', 'INSUFFANESTH', 'HRSAT', 'EXPCO2',
       'INTUBATION', 'BP', 'HR', 'PCWP', 'DISCONNECT', 'HISTORY', 'HREKG',
       'LVEDVOLUME', 'VENTMACH', 'ERRLOWOUTPUT', 'CATECHOL', 'VENTALV',
       'ANAPHYLAXIS', 'ARTCO2', 'PRESS', 'SHUNT', 'SAO2', 'ERRCAUTER',
       'LVFAILURE', 'HYPOVOLEMIA', 'FIO2', 'KINKEDTUBE', 'TPR', 'CO',
       'VENTLUNG', 'MINVOL', 'VENTTUBE', 'PULMEMBOLUS', 'PAP', 'HRBP', 'CVP'],
 

 10%|▉         | 65/666 [00:19<02:57,  3.39it/s] 


Valor do score K2: -106412.82447895246
Ordem das variáveis: Index(['MINVOLSET', 'STROKEVOLUME', 'PVSAT', 'INSUFFANESTH', 'HRSAT', 'EXPCO2',
       'INTUBATION', 'BP', 'HR', 'PCWP', 'DISCONNECT', 'HISTORY', 'HREKG',
       'LVEDVOLUME', 'VENTMACH', 'ERRLOWOUTPUT', 'CATECHOL', 'VENTALV',
       'ANAPHYLAXIS', 'ARTCO2', 'PRESS', 'SHUNT', 'SAO2', 'ERRCAUTER',
       'LVFAILURE', 'HYPOVOLEMIA', 'FIO2', 'KINKEDTUBE', 'TPR', 'CO',
       'VENTLUNG', 'MINVOL', 'VENTTUBE', 'PULMEMBOLUS', 'PAP', 'HRBP', 'CVP'],
      dtype='object')
Melhor modelo: DAG with 37 nodes and 61 edges
Estrutura da rede: [('STROKEVOLUME', 'CO'), ('STROKEVOLUME', 'HYPOVOLEMIA'), ('PVSAT', 'SAO2'), ('PVSAT', 'VENTLUNG'), ('PVSAT', 'INTUBATION'), ('INTUBATION', 'SHUNT'), ('INTUBATION', 'PRESS'), ('HR', 'HRBP'), ('HR', 'HRSAT'), ('HR', 'CO'), ('HR', 'CATECHOL'), ('HR', 'HREKG'), ('HR', 'BP'), ('HR', 'TPR'), ('DISCONNECT', 'VENTTUBE'), ('DISCONNECT', 'VENTMACH'), ('LVEDVOLUME', 'PCWP'), ('LVEDVOLUME', 'HYPOVOLEMIA'), ('LVEDV

-106412.82447895245
-106412.82447895246
-106412.82447895246
